# Writing TIFF multilayer file readable from PhotoShop

There is no straightforward way to write a multilayer TIFF file readable from PhotoShop.
So far, the attempts in the following code have been unsuccessful.
* [This is a short descriotion of PS TIFF format](https://www.amyspark.me/blog/posts/2021/11/14/reading-and-writing-tiff-psds.html)
* [This is a thread with attempts to write a PS-readable TIFF](https://github.com/imageio/imageio/issues/708)

The following example works:
* [Layered TIFF with psdtags](https://github.com/cgohlke/psdtags/blob/master/examples/layered_tiff.py)

In [1]:
import tifffile
import os

In [3]:
path = "../test/img-tif"
in_paths = [(path + "/000{:d}.tif".format(i)) for i in range(5)]
images = [tifffile.imread(p) for p in in_paths]
out_dir = "img-test"
if not os.path.exists(out_dir): os.makedirs(out_dir)
out_path = out_dir + "/multi.tif"
data_block = bytes('Adobe Photoshop Document Data Block', 'ascii') + b'\x00'
bim = bytes('8BIM', 'ascii')
layer = bytes('Layr', 'ascii')
with tifffile.TiffWriter(out_path) as tiff:
    for i, img in enumerate(images):
        img_bytes = img.tobytes()
        img_len = len(img_bytes)
        img_len_bytes = bytes(str(img_len), 'ascii')
        all_bytes = data_block + bim + layer + img_len_bytes
        byte_length = len(all_bytes) - len(data_block)
        bytes_bytes = bytes(byte_length)
        allll = all_bytes + bytes_bytes
        metad = { "37724": str(allll), 'LayerName': f'Layer_{i+1}',
                'DocumentName': 'Multilayer TIFF',
                'Software': 'Python tifffile' }
        tiff.write(img, photometric='rgb',
            contiguous=False, #planarconfig='separate',
            compression='none', #subfiletype=1 if i > 0 else 0,
            predictor=False,
            metadata=metad)